人工智能模拟人类思维过程的可能性，并不局限于被动性任务（比如目标识别）和大多数反应性任务（比如驾驶汽车），它还包括创造性活动。在不远的未来，我们所消费的大部分文化内容，其创造过程都将得到来自人工智能的实质性帮助。 的确，到目前为止，我们见到的人工智能艺术作品的水平还很低。人工智能还远远比不上人类编剧、画家和作曲家。但是，替代人类始终都不是我们要谈论的主题，人工智能不会替代我们自己的智能，而是会为我们的生活和工作带来更多的智能，即另一种类型的智能。在许多领域，特别是创新领域中，人类将会使用人工智能作为增强自身能力的工具，实现比人工智能更加强大的智能

### 8.1.1　生成式循环网络简史

截至 2014年年底，还没什么人见过  LSTM这一缩写，即使在机器学习领域也不常见。用循环网络生成序列数据的成功应用在 2016年才开始出现在主流领域。但是，这些技术都有着相当长的历史，最早的是 1997年开发的  LSTM算法。这一新算法早期用于逐字符地生成文本。

2002年，当时在瑞士  Schmidhuber实验室工作的 Douglas Eck首次将  LSTM应用于音乐生成，并得到了令人满意的结果。Eck现在是  Google Brain（谷歌大脑）的研究人员，2016年他在那里创建了一个名为 Magenta的新研究小组，重点研究将现代深度学习技术应用于制作迷人的音乐。有时候，好的想法需要 15年才能变成实践。

在 20世纪末和  21世纪初，Alex  Graves在使用循环网络生成序列数据方面做了重要的开创性工作。特别是他在 2013年的工作，利用笔触位置的时间序列将循环混合密度网络应用于生成类似人类的手写笔迹，有人认为这是一个转折点。在那个特定时刻，神经网络的这个具体应用中，能够做梦的机器这一概念适时地引起了我的兴趣，并且在我开始开发    Keras时为我提供了重要的灵感。Graves在  2013年上传到预印本服务器  arXiv上的  LaTeX文件中留下了一条类似的注释性评论：“序列数据生成是计算机所做的最接近于做梦的事情。”几年之后，我们将这些进展视作理所当然，但在当时看到 Graves的演示，很难不为其中所包含的可能性感到惊叹并受到启发。

从那以后，循环神经网络已被成功应用于音乐生成、对话生成、图像生成、语音合成和分子设计。它甚至还被用于制作电影剧本，然后由真人演员来表演。

### 8.1.2　如何生成序列数据

用深度学习生成序列数据的通用方法，就是使用前面的标记作为输入，训练一个网络（通常是循环神经网络或卷积神经网络）来预测序列中接下来的一个或多个标记。例如，给定输入the cat  is on the ma，训练网络来预测目标 t，即下一个字符。与前面处理文本数据时一样，标记（token）通常是单词或字符，给定前面的标记，能够对下一个标记的概率进行建模的任何网络都叫作语言模型（language  model）。语言模型能够捕捉到语言的潜在空间（latent space），即语言的统计结构。  

一旦训练好了这样一个语言模型，就可以从中采样（sample，即生成新序列）。向模型中输入一个初始文本字符串［即条件数据（conditioning  data）］，要求模型生成下一个字符或下一个单词（甚至可以同时生成多个标记），然后将生成的输出添加到输入数据中，并多次重复这一过程（见图 8-1）。这个循环可以生成任意长度的序列，这些序列反映了模型训练数据的结构，它们与人类书写的句子几乎相同。在本节的示例中，我们将会用到一个   LSTM层，向其输入从文本语料中提取的 N个字符组成的字符串，然后训练模型来生成第  N+1个字符。模型的输出是对所有可能的字符做 softmax，得到下一个字符的概率分布。这个 LSTM叫作字符级的神经语言模型（character-level neural language model）。

### 8.1.3　采样策略的重要性

生成文本时，如何选择下一个字符至关重要。一种简单的方法是贪婪采样（greedy sampling），就是始终选择可能性最大的下一个字符。但这种方法会得到重复的、可预测的字符串，看起来不像是连贯的语言。    

一种更有趣的方法是做出稍显意外的选择：在采样过程中引入随机性，即从下一个字符的概率分布中进行采样。这叫作随机采样（stochastic   sampling，stochasticity在这个领域中就是“随机”的意思）。在这种情况下，根据模型结果，如果下一个字符是 e的概率为0.3，那么你会有 30%的概率选择它。注意，贪婪采样也可以被看作从一个概率分布中进行采样，即某个字符的概率为 1，其他所有字符的概率都是 0。  

从模型的 softmax输出中进行概率采样是一种很巧妙的方法，它甚至可以在某些时候采样到不常见的字符，从而生成看起来更加有趣的句子，而且有时会得到训练数据中没有的、听起来像是真实存在的新单词，从而表现出创造性。但这种方法有一个问题，就是它在采样过程中无法控制随机性的大小。   

为什么需要有一定的随机性？考虑一个极端的例子——纯随机采样，即从均匀概率分布中抽取下一个字符，其中每个字符的概率相同。这种方案具有最大的随机性，换句话说，这种概率分布具有最大的熵。当然，它不会生成任何有趣的内容。再来看另一个极端——贪婪采样。贪婪采样也不会生成任何有趣的内容，它没有任何随机性，即相应的概率分布具有最小的熵。从“真实”概率分布（即模型  softmax函数输出的分布）中进行采样，是这两个极端之间的一个中间点。但是，还有许多其他中间点具有更大或更小的熵，你可能希望都研究一下。更小的熵可以让生成的序列具有更加可预测的结构（因此可能看起来更真实），而更大的熵会得到更加出人意料且更有创造性的序列。从生成式模型中进行采样时，在生成过程中探索不同的随机性大小总是好的做法。我们人类是生成数据是否有趣的最终判断者，所以有趣是非常主观的，我们无法提前知道最佳熵的位置。   

为了在采样过程中控制随机性的大小，我们引入一个叫作 softmax温度（softmax temperature）的参数，用于表示采样概率分布的熵，即表示所选择的下一个字符会有多么出人意料或多么可预测。给定一个 temperature值，将按照下列方法对原始概率分布（即模型的   softmax输出）    进行重新加权，计算得到一个新的概率分布。

In [ ]:
import numpy as np

def reweight_distribution(original_distribution, temperature =0.5): #original是概率值组成的一维Numpy数组，这些概率值之和为1
    distribution=np.log(original_distribution)/ temperature
    distribution=np.exp(distribution)
    return distribution/ np.sum(distribution) # 返回原始分布重新加权后的结果，因为此时distribution的求和可能不在等于1，因此需要将它除以求和

更高的温度得到的是熵更大的采样分布，会生成更加出人意料、更加无结构的生成数据，而更低的温度对应更小的随机性，以及更加可预测的生成数据（见图 8-2）。  更低的温度 =更确定，更高的温度 =更随机

### 8.1.4　实现字符级的 LSTM文本生成

面用 Keras来实现这些想法。首先需要可用于学习语言模型的大量文本数据。我们可以使用任意足够大的一个或多个文本文件——维基百科、《指环王》等。本例将使用尼采的一些作品，他是 19世纪末期的德国哲学家，这些作品已经被翻译成英文。因此，我们要学习的语言模型将是针对于尼采的写作风格和主题的模型，而不是关于英语的通用模型。

#### 8.1.4.1.准备数据

首先下载语料，并将其转换为小写。

代码清单 8-2　下载并解析初始文本文件

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
path = tf.keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower() # 转换成小写
print('Corpus length:', len(text))

606208/600901 [==============================] - 12s 19us/step
Corpus length: 600893



接下来，我们要提取长度为 maxlen的序列（这些序列之间存在部分重叠），对它们进行one-hot编码，然后将其打包成形状为(sequences,  maxlen, unique_characters)的三维Numpy数组。与此同时，我们还需要准备一个数组 y，其中包含对应的目标，即在每一个所提取的序列之后出现的字符（已进行 one-hot编码）。

In [4]:
maxlen =60 # 提取60个字符组成的序列
step= 3 # 每3个字符采样一个新序列
sentences=[] # 保存所提取的序列
next_chars=[] # 保存目标
for i in range(0,len(text)- maxlen, step):
    sentences.append(text[i: i+maxlen])
    next_chars.append(text[i+ maxlen])
    
print('Number of sequnces:', len(sentences))

chars= sorted(list(set(text))) # 语料中唯一字符(字母)组成的列表
print('Unique characters:', len(chars))
char_indices = dict((char, char.index(char)) for char in chars) # 一个字典，将唯一字符映射为它在列表chars中的索引

print('Vectorization...')
x= np.zeros((len(sentences),maxlen,len(chars)),dtype= np.bool) # 将字符编码转换成二进制数组
y= np.zeros((len(sentences),len(chars)),dtype= np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i,t,char_indices[char]]=1
    y[i,char_indices[next_chars[i]]]=1


Number of sequnces: 200278
Unique characters: 57
Vectorization...


<ipython-input-4-ec0dbeae7d4a>:16: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x= np.zeros((len(sentences),maxlen,len(chars)),dtype= np.bool)
<ipython-input-4-ec0dbeae7d4a>:17: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y= np.zeros((len(sentences),len(chars)),dtype= np.bool)


#### 8.1.4.2.构建网络

这个网络是一个单层LSTM，然后是一个Dense分类器和对所有可能字符的    softmax。但要注意，循环神经网络并不是序列数据生成的唯一方法，最近已经证明一维卷积神经网络也可以成功用于序列数据生成。

In [8]:
# 用于预测下一个字符的单层 LSTM模型

from tensorflow.keras import layers
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

#### 8.1.4.3.训练语言模型并从中采样

给定一个训练好的模型和一个种子文本片段，我们可以通过重复以下操作来生成新的文本。  
  
(1)给定目前已生成的文本，从模型中得到下一个字符的概率分布。

(2)根据某个温度对分布进行重新加权。

(3)根据重新加权后的分布对下一个字符进行随机采样。

(4)将新字符添加到文本末尾。

下列代码将对模型得到的原始概率分布进行重新加权，并从中抽取一个字符索引［采样函数（sampling function）］。

In [9]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
import random
import sys 
for epoch in range(1,2):
    print('epoch', epoch)
    model.fit(x,y,batch_size= 128, epochs=1) # 将模型在数据上拟合一次
    start_index= random.randint(0, len(text)-maxlen-1) # 随机选择一个文本种子
    generated_text =text[start_index: start_index+maxlen]
    print('--- Generating with seed:"' + generated_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]: # 尝试不同的采样温度
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled= np.zeros((1, maxlen,len(chars)))  # 对目前生成的字符进行one-hot编码
            for t, char in enumerate(generated_text):
                sampled[0,t,char_indices[char]]=1.
                
            preds= model.predict(sampled,verbose=0)[0]
            next_index= sample(preds, temperature)
            next_char= chars[next_index]
            
            generated_text += next_char
            generated_text =generated_text[1:]
            sys.stdout.write(next_char)
        

可见，较小的温度值会得到极端重复和可预测的文本，但局部结构是非常真实的，特别是所有单词都是真正的英文单词（单词就是字符的局部模式）。随着温度值越来越大，生成的文本也变得更有趣、更出人意料，甚至更有创造性，它有时会创造出全新的单词，听起来有几分可信（比如 eterned和  troveration）。对于较大的温度值，局部模式开始分解，大部分单词看起来像是半随机的字符串。毫无疑问，在这个特定的设置下，0.5的温度值生成的文本最为有趣。一定要尝试多种采样策略！在学到的结构与随机性之间，巧妙的平衡能够让生成的序列非常有趣。 

注意，利用更多的数据训练一个更大的模型，并且训练时间更长，生成的样本会比上面的结果看起来更连贯、更真实。但是，不要期待能够生成任何有意义的文本，除非是很偶然的情况。你所做的只是从一个统计模型中对数据进行采样，这个模型是关于字符先后顺序的模型。语言是一种信息沟通渠道，信息的内容与信息编码的统计结构是有区别的。为了展示这种区别，我们来看一个思想实验：如果人类语言能够更好地压缩通信，就像计算机对大部分数字通信所做的那样，那么会发生什么？语言仍然很有意义，但不会具有任何内在的统计结构，所以不可能像刚才那样学习一个语言模型。

### 8.1.5　小结 

我们可以生成离散的序列数据，其方法是：给定前面的标记，训练一个模型来预测接下来的一个或多个标记。  

对于文本来说，这种模型叫作语言模型。它可以是单词级的，也可以是字符级的。  

对下一个标记进行采样，需要在坚持模型的判断与引入随机性之间寻找平衡。‰处理这个问题的一种方法是使用   softmax温度。一定要尝试多种不同的温度，以找到合适的那一个。